In [1]:
import os
os.environ["JAX_PLATFORMS"]="cpu"
import sys
sys.path.append("..")
import yaml
import jax
import dataclasses

In [2]:
from flax_orbax import checkpointer, serialization

In [3]:
import flax.linen as nn
import optax
import flax.training.train_state
model = nn.Sequential([nn.Dense(10, kernel_init=nn.initializers.ones), nn.Dense(10, kernel_init=nn.initializers.ones)])
params = model.init(jax.random.key(0), jax.numpy.ones((1, 10)))['params']
tx = serialization.wrap(optax.adam)(1e-3)
state = flax.training.train_state.TrainState.create(
    apply_fn=model,
    params=params,
    tx=tx,
)

In [4]:
from flax_orbax import checkpointer

In [5]:
from absl import logging

# logging.set_verbosity(logging.DEBUG)  # Or INFO/WARNING

In [6]:
import orbax.checkpoint as ocp

path = ocp.test_utils.erase_and_create_empty('/tmp/my-checkpoints/')

In [7]:
ckptr = checkpointer.Checkpointer()
# 'checkpoint_name' must not already exist.
ckptr.save(path / 'checkpoint_name', state)
ckptr.wait_until_finished()

INFO:absl:orbax-checkpoint version: 0.11.15
DEBUG:absl:Created BasePyTreeCheckpointHandler: use_ocdbt=True, use_zarr3=False, pytree_metadata_options=PyTreeMetadataOptions(support_rich_types=False), array_metadata_store=<orbax.checkpoint._src.metadata.array_metadata_store.Store object at 0x7fcee78efa00>, enable_pinned_host_transfer=False
INFO:absl:[thread=MainThread] Failed to get flag value for EXPERIMENTAL_ORBAX_USE_DISTRIBUTED_PROCESS_ID.
INFO:absl:[process=0][thread=MainThread] Using barrier_sync_fn: <function get_barrier_sync_fn.<locals>.<lambda> at 0x7fced47e5e10> timeout: 600 secs and primary_host=0 for async checkpoint writes
DEBUG:absl:[process=0][thread=MainThread] Skipping global process sync, barrier name: _Checkpointer:save_start
DEBUG:absl:[process=0][thread=MainThread] Skipping global process sync, barrier name: _next_awaitable_signal_operation_id:sync
DEBUG:absl:[process=0] Synchronized next awaitable signal operation id to 1
INFO:absl:[process=0] Started async saving ch

In [8]:
ckptr.restore(path / 'checkpoint_name/')

INFO:absl:Restoring checkpoint from /tmp/my-checkpoints/checkpoint_name.
DEBUG:absl:Reading pytree metadata file: /tmp/my-checkpoints/checkpoint_name/_METADATA with pytree_metadata_options: PyTreeMetadataOptions(support_rich_types=False)
DEBUG:absl:Created InternalTreeMetadata with pytree_metadata_options=PyTreeMetadataOptions(support_rich_types=False), has tree_metadata_entries=15, has rich typed value_metadata_tree=False
DEBUG:absl:directory=/tmp/my-checkpoints/checkpoint_name, restore_args={'opt_state': [{'count': ArrayRestoreArgs(restore_type=<class 'jax.Array'>, dtype=dtype('int32'), mesh=None, mesh_axes=None, sharding=SingleDeviceShardingMetadata(device_str=TFRT_CPU_0), global_shape=(), shape=(), strict=True), 'mu': {'layers_0': {'bias': ArrayRestoreArgs(restore_type=<class 'jax.Array'>, dtype=dtype('float32'), mesh=None, mesh_axes=None, sharding=SingleDeviceShardingMetadata(device_str=TFRT_CPU_0), global_shape=(10,), shape=(10,), strict=True), 'kernel': ArrayRestoreArgs(restore_

{'opt_state': [{'count': Array(0, dtype=int32),
   'mu': {'layers_0': {'bias': Array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32),
     'kernel': Array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
            [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
            [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
            [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
            [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
            [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
            [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
            [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
            [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
            [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.]], dtype=float32)},
    'layers_1': {'bias': Array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0.], dtype=float32),
     'kernel': Array([[0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
            [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
            [0., 0., 0., 0., 0., 0., 0., 0., 0., 0.],
            [0., 0., 0., 0.,

In [9]:
ckptr.metadata(path / "checkpoint_name")

DEBUG:absl:Reading pytree metadata file: /tmp/my-checkpoints/checkpoint_name/_METADATA with pytree_metadata_options: PyTreeMetadataOptions(support_rich_types=False)
DEBUG:absl:Created InternalTreeMetadata with pytree_metadata_options=PyTreeMetadataOptions(support_rich_types=False), has tree_metadata_entries=15, has rich typed value_metadata_tree=False


TreeMetadata(
  custom_metadata=None
  tree={'opt_state': [{'count': ArrayMetadata :  name=opt_state.0.count,  directory=/tmp/my-checkpoints/checkpoint_name,  shape=(),  sharding=SingleDeviceShardingMetadata(device_str=TFRT_CPU_0),  dtype=int32,  storage=StorageMetadata(chunk_shape=(), write_shape=()),, 'mu': {'layers_0': {'bias': ArrayMetadata :  name=opt_state.0.mu.layers_0.bias,  directory=/tmp/my-checkpoints/checkpoint_name,  shape=(10,),  sharding=SingleDeviceShardingMetadata(device_str=TFRT_CPU_0),  dtype=float32,  storage=StorageMetadata(chunk_shape=(10,), write_shape=(10,)),, 'kernel': ArrayMetadata :  name=opt_state.0.mu.layers_0.kernel,  directory=/tmp/my-checkpoints/checkpoint_name,  shape=(10, 10),  sharding=SingleDeviceShardingMetadata(device_str=TFRT_CPU_0),  dtype=float32,  storage=StorageMetadata(chunk_shape=(10, 10), write_shape=(10, 10)),}, 'layers_1': {'bias': ArrayMetadata :  name=opt_state.0.mu.layers_1.bias,  directory=/tmp/my-checkpoints/checkpoint_name,  shape=(

In [10]:
ocp

<module 'orbax.checkpoint' from '/home/dibya/nfs/libraries/flax_orbax/.venv/lib/python3.10/site-packages/orbax/checkpoint/__init__.py'>

In [11]:
options = ocp.CheckpointManagerOptions()
mngr = ocp.CheckpointManager(
  ocp.test_utils.erase_and_create_empty('/tmp/ckpt1/'),
  checkpointer.Checkpointer(),
  options=options,
)

mngr.save(0, state)
mngr.wait_until_finished()

mngr.restore(0)

DEBUG:absl:Created BasePyTreeCheckpointHandler: use_ocdbt=True, use_zarr3=False, pytree_metadata_options=PyTreeMetadataOptions(support_rich_types=False), array_metadata_store=<orbax.checkpoint._src.metadata.array_metadata_store.Store object at 0x7fcee78efa00>, enable_pinned_host_transfer=False
INFO:absl:[process=0][thread=MainThread] Using barrier_sync_fn: <function get_barrier_sync_fn.<locals>.<lambda> at 0x7fced47e7370> timeout: 600 secs and primary_host=0 for async checkpoint writes
INFO:absl:[process=0][thread=MainThread] CheckpointManager init: checkpointers=<flax_orbax.checkpointer.Checkpointer object at 0x7fcfbc40fa90>, item_names=None, item_handlers=None, handler_registry=None
INFO:absl:Deferred registration for item: "default". Adding handler `<flax_orbax.checkpointer.CheckpointHandler object at 0x7fced47a8160>` for item "default" and save args `<class 'flax_orbax.checkpointer.SaveArgs'>` and restore args `<class 'flax_orbax.checkpointer.SaveArgs'>` to `_handler_registry`.
INF

ValueError: Attempted to construct default restoration arguments for item "default", but the `CheckpointArgs` class of type <class 'flax_orbax.checkpointer.SaveArgs'> is not default constructible. Original error: StandardSaveArgs.__init__() missing 1 required positional argument: 'item'.